In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

import numpy as np

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 407054468304957562
]



## Dataflow

Датафлоу - концепция программирования: программа или модель представляется в форме направленного графа (_вычислений_).

Такой подход обладает следущими преимуществами: 
* Простота параллелизации программы: по графу легко понять, какие операции можно выполнять одновременно
* Распределенные вычисления (кластеры видеокарт, CPU, TPU)
* Компиляция графа: создается быстрые оптимизированный код для вычислений
* Граф вычислений - универсальное представление, которое является портируемым между различными языками и платформами

### Мы будет говорить об интерфейсе на python

### Граф

В python представлен классом `tf.Graph`

У графа есть следующие "основные" составляющие:
* структура графа - ребра и узлы (edges и nodes)
* коллекции, связанные с графом (подробности далее)

### Узлы и ребра

* Узлы графа - это операции `tf.Operation`
* Ребра графа - значения, представленные наследниками класса `tf.Tensor`


#### Добавление значений в граф

* Базовый "кирпичик" - функция `tf.constant(x)`, или операция, всегда возвращающая x. 

> Например, операция `tf.constant(13)` создает `tf.Tensor` (ребро) со значением $13$

* Другой базовый элемент - `tf.Variable(x)`, создающий узел, в котором хранится _изменяемое_ значение. Это может быть полезно, например, при обучении модели: в переменной будут храниться веса модели. 
* Над тензорами можно проводить операции, создавая новые узлы.
* Для оптимизации нужно вызвать `tf.train.Optimizer.minimize` - и ко всем операциям в графе будут добавлены операции (и связанные с ними тензоры), вычисляющие градиенты.


#### Пример



In [3]:
a = tf.constant(2)  # Создаем узлы графа c константами
b = tf.constant(2)  
c = a + b  # Складываем значения - создаем новый узел

print(a)
print(b)
print(c)

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)


In [4]:
print(a)
print(b)
print(c)

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)


## Создаем сессию и сохраняем результаты 

> Скоро разберемся с сессиями и tensorboard

In [5]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)  # logs - имя директории, где будут храниться результаты
    print(sess.run(c))  # Получаем результат вычислений в сессии
    writer.close()

4



#### Пример графа
<img src="files/img/simple_graph.png">

#### Обозначения
<img src="files/img/legend.png" width="400">




### tf.Session

> "Просто так" значения в графе не вычисляются, нужно создавать и запускать сессию, чтобы получить результаты

Несколько фактов:
* Класс `tf.Session`
* Сайт tensorflow сообщает, что "сессия инкапсулирует окружение, в котором выполняются `tf.Operation` и вычисляются значения `tf.Tensor`"
* Можно активировать eager mode, в котором вычисления осуществляются "на лету", тогда `tf.Session` не нужен
* В tensorflow 2.0 от `tf.Session` отказались 

#### Как корректно использовать tf.Session

> Не сосвсем корректно:


In [6]:
a = tf.constant(1)
b = tf.constant(2)
c = a + b

sess = tf.Session()
sess.run(c)

3


Чего не хватает?

Сессия может "захватывать" ресурсы, после завершения вычислений их надо освободить. Есть два способа:

> * `sess.close()` 
* `with tf.Session() as sess: ...`
* Второй способ закрывает сессию автоматически и предпочтительнее


In [7]:
sess.close()

try:
    print(sess.run(c))
except RuntimeError:
    print("Session is closed")

Session is closed


In [8]:
a = tf.constant(1)
b = tf.constant(2)
c = a + b

with tf.Session() as sess:
    result = sess.run(c)

print(result)

try:
    sess.run(c)
except RuntimeError:
    print("Session is closed")

3
Session is closed



### Скоупы и имена

Важная часть работы tensorflow - скоупы (scope) и имена переменных. Используются для удобства работы с графом

#### Имена переменных

Все операции, создающие новые операции (`tf.Operation`) или новый `tf.Tensor` - могут получить имя: 

> `zero = tf.constant(0, name='zero') `
    
Имена не работают в _eager mode_!

Повторяющиеся имена tensorflow "за вас" делает различимыми:



In [9]:
zero_1 = tf.constant(0, name='zero')
zero_1.name

'zero:0'

    
## Scope

("Рамки"? Области действия?)

> Нужны для группировки переменных и тензоров. В общем, для наведения порядка в том коде, который вы пишите.

* Скоупы организованы иерархически, как вложенные директории
* Имена в разных скоупах могут повторяться (как имена файлов во вложеннх директориях)

_Добавим еще две переменных внутри скоупов к графу_:



In [10]:
with tf.name_scope('outer_scope'):
    zero_outer = tf.constant(0, name='zero')
    
    with tf.name_scope('inner_scope'):
        inner_scope = tf.constant(0, name='zero')    

In [11]:
# tf.get_default_graph().as_graph_def() - граф, представленный как JSON
for node in tf.get_default_graph().as_graph_def().node:
    print(node.name)

Const
Const_1
add
Const_2
Const_3
add_1
Const_4
Const_5
add_2
zero
outer_scope/zero
outer_scope/inner_scope/zero


<font size="4">
    
Разберем выражение по строкам:
    
</font>

In [12]:
print(f"Граф по умолчанию:\n {tf.get_default_graph()}")

def_graph = tf.get_default_graph()
print(f"Проверка типа: точно ли это tf.Graph?\n {isinstance(def_graph, tf.Graph)}")

Граф по умолчанию:
Проверка типа: точно ли это tf.Graph?
 True


    
Полезная команда: 

`tf.reset_default_graph()`

* "Сбрасывает" все, что есть в графе по умолчанию
* Создает новый граф, пустой
* Старый граф удаляется, при этом освобождается память. 
  * Освобождение памяти полезно, особенно при ограниченных ресурсах (например, GPU), и работе в ноутбуках
  * Можно "случайно" потерять обученную модель, поэтому не стоит делать reset необдуманно


In [13]:
tf.reset_default_graph()

In [14]:
for node in tf.get_default_graph().as_graph_def().node:
    print(node.name)

In [15]:
print(f"Граф по умолчанию:\n {tf.get_default_graph()}")

def_graph = tf.get_default_graph()
print(f"Проверка типа: точно ли это tf.Graph?\n {isinstance(def_graph, tf.Graph)}")

Граф по умолчанию:
Проверка типа: точно ли это tf.Graph?
 True


<font size="4">
    
Обратите внимание на то, что адреса графов в памяти до и после `tf.reset_default_graph()` различаются!

### Применение tensorboard

> Не забываем чистить граф при необходимости!

</font>

In [16]:
tf.reset_default_graph()

In [17]:
a = tf.constant(2, name='a')
b = tf.constant(2, name='b')
c = tf.add(a, b, name="a_plus_b")

In [18]:
print(a)
print(b)
print(c)

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    print(f"a + b = {sess.run(c)}")
    writer.close()

Tensor("a:0", shape=(), dtype=int32)
Tensor("b:0", shape=(), dtype=int32)
Tensor("a_plus_b:0", shape=(), dtype=int32)
a + b = 4


<font size="4">

### Граф вычисления a + b

<img src="files/img/a_plus_b.png" width="400">

### Как посмотреть граф вычислений?

Для этого существует tensorboard (устанавливается вместе с tensorflow), который реализует веб-интерфейс для изучения результатов вычислений.

* По умолчанию запускается на порте 6006
* Результаты экспериментов пишет `tf.summary.FileWriter(dirname, sess.graph)` в директорию `<dirname>`
* tensorboard запускается через командную строку. Нужно указывать директорию с логами `FileWriter`:
> `tensorboard --logdir=<dirname>`
* tensorboard работает с логами, и после `tf.reset_default_graph()` реузльтаты не пропадают

</font>

<font size="4">

## Подробнее о переменных

tensorflow - слишком сложный интерфейс для "просто" математических операций. Более интересная вещь - переменные, `tf.Variable`


Предположим, что мы хотим найти минимум функции $f(x) = 2x^2 + 5x - 9$. Как записать эту функцию на tensorflow?


</font>

In [19]:
tf.reset_default_graph()
a = tf.constant(2., name='a')
b = tf.constant(5., name='b')
c = tf.constant(-9., name='c')

<font size="4">

x - ?  
f(x) - ???

> `x` удобно представить с помощью переменной: `x = tf.Variable(<initial-value>, name=<name>)`



</font>

In [20]:
x = tf.Variable(0, name='x', dtype='float32')

In [21]:
f_x = a*x**2 + b * x + c

In [22]:
f_x

<tf.Tensor 'add_1:0' shape=() dtype=float32>

In [23]:
for node in tf.get_default_graph().as_graph_def().node:
    print(node.name)

a
b
c
x/initial_value
x
x/Assign
x/read
pow/y
pow
mul
mul_1
add
add_1


In [24]:
opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
sgd_operation = opt.minimize(loss=f_x)

In [25]:
for node in tf.get_default_graph().as_graph_def().node:
    print(node.name)

a
b
c
x/initial_value
x
x/Assign
x/read
pow/y
pow
mul
mul_1
add
add_1
gradients/Shape
gradients/grad_ys_0
gradients/Fill
gradients/add_1_grad/tuple/group_deps
gradients/add_1_grad/tuple/control_dependency
gradients/add_1_grad/tuple/control_dependency_1
gradients/add_grad/tuple/group_deps
gradients/add_grad/tuple/control_dependency
gradients/add_grad/tuple/control_dependency_1
gradients/mul_grad/Mul
gradients/mul_grad/Mul_1
gradients/mul_grad/tuple/group_deps
gradients/mul_grad/tuple/control_dependency
gradients/mul_grad/tuple/control_dependency_1
gradients/mul_1_grad/Mul
gradients/mul_1_grad/Mul_1
gradients/mul_1_grad/tuple/group_deps
gradients/mul_1_grad/tuple/control_dependency
gradients/mul_1_grad/tuple/control_dependency_1
gradients/pow_grad/Shape
gradients/pow_grad/Shape_1
gradients/pow_grad/BroadcastGradientArgs
gradients/pow_grad/mul
gradients/pow_grad/sub/y
gradients/pow_grad/sub
gradients/pow_grad/Pow
gradients/pow_grad/mul_1
gradients/pow_grad/Sum
gradients/pow_grad/Reshape
g

In [26]:
def reset_and_make_variables_part():
    """Для демонстрации одним блоком, без оптимизатора"""
    tf.reset_default_graph()
    
    # with tf.name_scope(name='f_x'):
    a = tf.constant(2., name='a')
    b = tf.constant(5., name='b')
    c = tf.constant(-9., name='c')
    x = tf.Variable(0, name='x', dtype='float32')
    f_x = a*x**2 + b * x + c
    
    return x, f_x


def reset_and_make_variables_example():
    """Для демонстрации одним блоком"""
    tf.reset_default_graph()

    # with tf.name_scope(name='f_x'):
    a = tf.constant(2., name='a')
    b = tf.constant(5., name='b')
    c = tf.constant(-9., name='c')
    x = tf.Variable(0, name='x', dtype='float32')
    f_x = a*x**2 + b * x + c
    f_x_summary = tf.summary.scalar('f_x_value', f_x)  # Пояснения дальше в разделе про tensorboard
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    sgd_operation = opt.minimize(loss=f_x)
    
    return x, f_x, sgd_operation, f_x_summary

### Возможные проблемы и их решение

In [27]:
try:
    with tf.Session() as sess:
        print(f"Show x: {sess.run(sgd_operation)}")
except tf.errors.FailedPreconditionError as e:
    print(e)

Attempting to use uninitialized value x
	 [[node x/read (defined at <ipython-input-20-d6c8f8a34f66>:1) ]]

Caused by op 'x/read', defined at:
  File "C:\Anaconda3\envs\tf1\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\envs\tf1\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\envs\tf1\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Anaconda3\envs\tf1\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Anaconda3\envs\tf1\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Anaconda3\envs\tf1\lib\asyncio\base_events.py", line 442, in run_forever
    self._run_once()
  File "C:\Anaconda3\envs\tf1\lib\asyncio\base_events.

In [28]:
x, f_x, sgd_operation, _ = reset_and_make_variables_example()
try:
    with tf.Session() as sess:
        print(f"Show x: {sess.run(x)}")
except tf.errors.FailedPreconditionError as e:
    print(e)

Attempting to use uninitialized value x
	 [[{{node _retval_x_0_0}}]]


<font size="4">


#### Пояснение к ошибке - инициализация переменных
В tensorflow переменные необходимо _инициализировать_. К сожалению, проставление _initial value_ не приводит к инициализации автоматически - и эту операцию необходимо проводить явно
    
</font>

In [29]:
x, f_x, sgd_operation, _ = reset_and_make_variables_example()

try:
    with tf.Session() as sess:
        sess.run(x.initializer)  # Здесь x инициализируется
        print(f"Show x: = {sess.run(x)}")
except tf.errors.FailedPreconditionError as e:
    print(e)

Show x: = 0.0


<font size="4">
    
### Альтернативная инициализация

> Все переменные, требующие инициализации, можно заранее "привязать" к одной операции, которая будет вызвана внутри сессии. Это удобнее, если требуется инициализировать _много_ переменных:

</font>

In [30]:
x, f_x, sgd_operation, _ = reset_and_make_variables_example()
init_op = tf.global_variables_initializer()  # Здесь x инициализируется

try:
    with tf.Session() as sess:
        sess.run(init_op)
        print(f"Show x: = {sess.run(x)}")
except tf.errors.FailedPreconditionError as e:
    print(e)

Show x: = 0.0


### Сначала посмотрим на граф функции без градиентов

In [31]:
x, f_x = reset_and_make_variables_part()
init_op = tf.initialize_all_variables() 

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    print(f"Show x: = {sess.run(x)}")
    writer.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Show x: = 0.0


### Граф
<img src='files/img/f_x.png' width=400>

### Инициализация x
<img src='files/img/x_init.png' width=650>

### Добавим градиент

In [32]:
x, f_x, sgd_operation, _ = reset_and_make_variables_example()
init_op = tf.initialize_all_variables() 

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    print(f"Show x: = {sess.run(x)}")
    print(f"Show f(x): = {sess.run(f_x)}")

    sess.run(sgd_operation)
    print('-------------------')
    print(f"Show x: = {sess.run(x)}")
    print(f"Show f(x): = {sess.run(f_x)}")

    writer.close()

Show x: = 0.0
Show f(x): = -9.0
-------------------
Show x: = -0.5
Show f(x): = -11.0


### Добавлен скоуп с градиентами
<img src='files/img/f_x_with_grad.png' width=650>

### Содержимое скоупа
<img src='files/img/f_x_gradients.png' width=650>

### Дополнительные графы
<img src='files/img/f_x_with_grad_aux.png' width=650>

### Граф SGD
<img src='files/img/sgd.png' width=650>

In [33]:
iterations = 100
x, f_x, sgd_operation, f_x_summary = reset_and_make_variables_example()
init_op = tf.initialize_all_variables() 

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    for step in range(iterations): 
        sess.run(sgd_operation)
        
    print(f"Show x: = {sess.run(x)}")
    print(f"Show f(x): = {sess.run(f_x)}")


    writer.close()

Show x: = -1.2499998807907104
Show f(x): = -12.125


### Добавим отслеживание значения функции

In [34]:
iterations = 100
x, f_x, sgd_operation, f_x_summary = reset_and_make_variables_example()
init_op = tf.initialize_all_variables() 

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    for step in range(iterations): 
        sgd, f_x_step =  sess.run([sgd_operation, f_x_summary])
        writer.add_summary(f_x_step, step)
        
    print(f"Show x: = {sess.run(x)}")
    print(f"Show f(x): = {sess.run(f_x)}")

    writer.close()

Show x: = -1.2499998807907104
Show f(x): = -12.125


## Градиент (the hard way)

In [41]:
def reset_and_make_gradients():
    """Для демонстрации одним блоком"""
    tf.reset_default_graph()

    # with tf.name_scope(name='f_x'):
    a = tf.constant(2., name='a')
    b = tf.constant(5., name='b')
    c = tf.constant(-9., name='c')
    x = tf.Variable(0, name='x', dtype='float32')
    f_x = a*x**2 + b * x + c
    f_x_summary = tf.summary.scalar('f_x_value', f_x)  # Пояснения дальше в разделе про tensorboard
    grads = tf.gradients(f_x, [x])
    
    return x, f_x, grads, f_x_summary

In [42]:
iterations = 100
x, f_x, grad, f_x_summary = reset_and_make_gradients()
init_op = tf.initialize_all_variables() 

In [44]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    
    grad_step, x_val, f_x_step =  sess.run([grad, x, f_x_summary])
    writer.add_summary(f_x_step, step)
        
    print(f"Show grad f_x w.r.t. x at {x_val}: = {grad_step}")
    print(f"f_x_step = {f_x_step}")
    writer.close()

Show grad f_x w.r.t. x at 0.0: = [5.0]
f_x_step = b'\n\x10\n\tf_x_value\x15\x00\x00\x10\xc1'


In [45]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    
    grad_val, x_val, f_x_step =  sess.run([grad, x, f_x_summary])
    writer.add_summary(f_x_step, step)
        
    print(f"Show grad f_x w.r.t. x at {x_val}: = {grad_val}")
    writer.close()

Show grad f_x w.r.t. x at 0.0: = [5.0]


<img src="files/img/f_x_just_grads.png" width=650>

## "Свой" градиентный спуск

**Точнее, градиентный спуск без стандартного оптимизатора SGD**

In [46]:
x, f_x, grad, f_x_summary = reset_and_make_gradients()
init_op = tf.initialize_all_variables() 

iterations = 50
learning_rate = 0.1

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    for step in range(iterations):
        grad_val, x_val, f_x_step =  sess.run([grad, x, f_x_summary])  # [1]
        grad_step = learning_rate * grad_val[0]  # [2]
        update = tf.assign(x, x_val - grad_step)  # [3]
        sess.run(update)  # [4]
        
        print(f"Show grad f_x w.r.t. x at {x_val}: = {grad_val}")
        
    writer.close()

Show grad f_x w.r.t. x at 0.0: = [5.0]
Show grad f_x w.r.t. x at -0.5: = [3.0]
Show grad f_x w.r.t. x at -0.800000011920929: = [1.8]
Show grad f_x w.r.t. x at -0.9800000190734863: = [1.0799999]
Show grad f_x w.r.t. x at -1.0880000591278076: = [0.64799976]
Show grad f_x w.r.t. x at -1.1528000831604004: = [0.38879967]
Show grad f_x w.r.t. x at -1.1916800737380981: = [0.2332797]
Show grad f_x w.r.t. x at -1.215008020401001: = [0.13996792]
Show grad f_x w.r.t. x at -1.2290048599243164: = [0.08398056]
Show grad f_x w.r.t. x at -1.2374029159545898: = [0.050388336]
Show grad f_x w.r.t. x at -1.2424417734146118: = [0.030232906]
Show grad f_x w.r.t. x at -1.2454650402069092: = [0.01813984]
Show grad f_x w.r.t. x at -1.2472790479660034: = [0.010883808]
Show grad f_x w.r.t. x at -1.248367428779602: = [0.006530285]
Show grad f_x w.r.t. x at -1.2490204572677612: = [0.003918171]
Show grad f_x w.r.t. x at -1.2494122982025146: = [0.0023508072]
Show grad f_x w.r.t. x at -1.2496473789215088: = [0.001410

<font size="4">

Действия по пунктам:

* [1] вычисление значений градиента, $x$ и значения $f(x)$
* [2] расчет шага градиента с учетом learning rate'a
* [3] создание операции обновления $x$
* [4] обновление $x$ при помощи `sess.run()`
    

</font>

-- предполагаю, что примерно здесь закончится первый час --

<font size="4">

# Логистическая регрессия в tensorflow


</font>

In [47]:
tf.reset_default_graph()
gl_norm_initializer = tf.glorot_normal_initializer()

input_shape = 4  # [1]
output_shape = 1  # [2]

with tf.name_scope('model'):
    weights = tf.Variable(gl_norm_initializer((input_shape, 1)), name='weights')
    bias = tf.Variable(gl_norm_initializer((output_shape, 1)))    

<font size="4">
    
### Как "поместить" данные внутрь модели?

> Для "размещения" данных в tensorflow есть (фабрика?) `tf.placeholder`

</font>

In [49]:
tf.reset_default_graph()
gl_norm_initializer = tf.glorot_normal_initializer()

input_shape = 4  # [1]
output_shape = 1


with tf.name_scope('communications'):
    data = tf.placeholder(dtype=tf.float32, shape=[None, input_shape])  # [2]
    target = tf.placeholder(dtype=tf.float32, shape=[None, output_shape])
    
    
with tf.name_scope('model'):
    weights = tf.Variable(gl_norm_initializer((input_shape, 1)), name='weights')
    bias = tf.Variable(gl_norm_initializer((output_shape, 1)))    
    model = tf.matmul(data, weights) + bias  # [3]

<font size="4">

Действия по пунктам:

* [1] переменные, в которых хранятся размерностит модели
* [2] создание плейсхолдера; `None` - неопределенный (изменяемый) размер
* [3] создание модели
* Плейсхолдеры не нужно инициализировать; но в них нужно передавать данные (feed)


#### Оптимизация модели

Для оптимизации модели можно написать собственные формулы - взяв за основу 

$L(\hat{y}, y) = - \sum\limits_{i=1}^{n} \hat{y} \cdot \log{y} + (1 - \hat{y}) \log{(1 - y)}$

проще воспользоваться набором готовых:
   
`loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model, labels=target))`

* `tf.reduce_mean` - вычисление среднего значения тензора
* `tf.nn.sigmoid_cross_entropy_with_logits` - бинарная кросс-энтропия, применяющая сигмоиду к логитам ($wx + b$)
</font>

In [51]:
tf.reset_default_graph()
gl_norm_initializer = tf.glorot_normal_initializer()

input_shape = 784
output_shape = 1
learning_rate = 0.03  # Добавили learning rate


with tf.name_scope('communications'):
    data = tf.placeholder(dtype=tf.float32, shape=[None, input_shape])
    target = tf.placeholder(dtype=tf.float32, shape=[None, output_shape])
    
    
with tf.name_scope('model'):
    weights = tf.Variable(gl_norm_initializer((input_shape, 1)), name='weights')
    bias = tf.Variable(gl_norm_initializer((output_shape, 1)))    
    model = tf.matmul(data, weights) + bias 
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model, labels=target))
    opt = tf.train.GradientDescentOptimizer(learning_rate)  # [1]
    goal = opt.minimize(loss)  # [2]
    
with tf.name_scope('evaluate'):
    prediction = tf.round(tf.sigmoid(model))  # [3]
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, target),  # [4]
                              dtype=tf.float32), 
                             )
    
init_op = tf.initialize_all_variables() 


Действия по пунктам:

* [1] создание оптимизатора
* [2] добавление лосса в функции, которые необходимо минимизировать
* [3] операция вычисления значений (т.е. вероятностей в данном случае)
* [4] расчет точности модели: подсчет количества совпадений прогноза и целевого значения





## Подготовка данных и запуск

> tensorflow не умеет "удобно" подготавливать данные. Есть следующие подходы:

* Подготовка данных в pandas / numpy / другое
* tensorflow_transform, apache beam



# Минимальная подготовка данных

In [23]:
from tensorflow.keras.datasets import mnist


def prepare_data(l1=1, l2=7):
    (train_X, train_y), (test_X, test_y) = mnist.load_data()

    train_X = train_X[(train_y == l1) | (train_y == l2)]
    train_y = train_y[(train_y == l1) | (train_y == l2)]

    test_X = test_X[(test_y == l1) | (test_y == l2)]
    test_y = test_y[(test_y == l1) | (test_y == l2)]

    train_X = (train_X / (train_X.max() / 2) - 1).reshape((-1, 784))
    test_X = (test_X / (test_X.max() / 2) - 1).reshape((-1, 784))
    
    train_y = np.array([0 if i == l1 else 1 for i in train_y])
    test_y = np.array([0 if i == l1 else 1 for i in test_y])
    
    return train_X, train_y, test_X, test_y


def prepare_data_for_nn():
    (train_X, train_y), (test_X, test_y) = mnist.load_data()

    train_X = (train_X / (train_X.max() / 2) - 1).reshape((-1, 784))
    test_X = (test_X / (test_X.max() / 2) - 1).reshape((-1, 784))

    return train_X, train_y, test_X, test_y

# Слои

## Полносвязный слой

> В tensorflow 1.13 было несколько вариантов API для работы со слоями. 

* `tf.layers`
* `tf.keras.layers`

В версии 2.0 осталось только keras API, но для знакомства и возможности работы с legacy-кодом посмотрим на модуль `tf.layers`. Однако новый код рекомендую писать на tf 2.0.

## tf.layers API


In [21]:
tf.reset_default_graph()
gl_norm_initializer = tf.glorot_normal_initializer()

input_shape = 784
output_shape = 1
learning_rate = 0.03  # Добавили learning rate


with tf.name_scope('communications'):
    data = tf.placeholder(dtype=tf.float32, shape=[None, input_shape])
    target = tf.placeholder(dtype=tf.float32, shape=[None, output_shape])
    
    
with tf.name_scope('model'):
    weights = tf.Variable(gl_norm_initializer((input_shape, 1)), name='weights')
    bias = tf.Variable(gl_norm_initializer((output_shape, 1)))    
    model = tf.matmul(data, weights) + bias 
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model, labels=target))
    opt = tf.train.GradientDescentOptimizer(learning_rate)  # [1]
    goal = opt.minimize(loss)  # [2]
    
with tf.name_scope('evaluate'):
    prediction = tf.round(tf.sigmoid(model))  # [3]
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, target),  # [4]
                              dtype=tf.float32), 
                             )
    
init_op = tf.initialize_all_variables() 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [24]:
train_X, train_y, test_X, test_y = prepare_data()

batch_size = 32
iter_num = 300

loss_trace, train_acc, test_acc = [], [], []

with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(init_op)
    
    for epoch in range(iter_num):

        batch_index = np.random.choice(len(train_X), size=batch_size)
        batch_train_X = train_X[batch_index]
        batch_train_y = np.matrix(train_y[batch_index]).T
        
        sess.run(goal, 
                 feed_dict={data: batch_train_X,
                            target: batch_train_y,
                           },
                )
        temp_loss = sess.run(loss, 
                             feed_dict={data: batch_train_X, 
                                        target: batch_train_y,
                                       },
                            )
        

        
        loss_trace.append(temp_loss)


        if (epoch + 1) % 10 == 0:
           
            temp_train_acc = sess.run(accuracy,
                                      feed_dict={data: train_X, 
                                                 target: np.matrix(train_y).T,
                                                },
                                     )
            temp_test_acc = sess.run(accuracy, 
                                     feed_dict={data: test_X, 
                                                target: np.matrix(test_y).T,
                                               },
                                    )
            
            train_acc.append(temp_train_acc)
            test_acc.append(temp_test_acc)
            
            print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'
                  .format(epoch + 1, temp_loss, temp_train_acc, temp_test_acc))
            
    writer.close()

epoch:   10 loss: 0.109232 train_acc: 0.974706 test_acc: 0.972261
epoch:   20 loss: 0.135691 train_acc: 0.978166 test_acc: 0.975497
epoch:   30 loss: 0.086622 train_acc: 0.979473 test_acc: 0.976884
epoch:   40 loss: 0.056616 train_acc: 0.977551 test_acc: 0.973648
epoch:   50 loss: 0.073894 train_acc: 0.984162 test_acc: 0.982432
epoch:   60 loss: 0.074428 train_acc: 0.980780 test_acc: 0.977346
epoch:   70 loss: 0.076377 train_acc: 0.986315 test_acc: 0.984281
epoch:   80 loss: 0.048938 train_acc: 0.985316 test_acc: 0.983356
epoch:   90 loss: 0.062392 train_acc: 0.976628 test_acc: 0.969949
epoch:  100 loss: 0.050462 train_acc: 0.983855 test_acc: 0.980583
epoch:  110 loss: 0.018965 train_acc: 0.984009 test_acc: 0.981507
epoch:  120 loss: 0.017555 train_acc: 0.984162 test_acc: 0.981045
epoch:  130 loss: 0.011449 train_acc: 0.978089 test_acc: 0.972723
epoch:  140 loss: 0.021601 train_acc: 0.987776 test_acc: 0.984281
epoch:  150 loss: 0.030409 train_acc: 0.987930 test_acc: 0.984281
epoch:  16

# Keras API

In [26]:
import tensorflow.keras as keras
from tensorflow.keras import layers
# from tqdm import tqdm


# Get the model.

tf.reset_default_graph()


train_X, train_y, test_X, test_y = prepare_data_for_nn()


input_shape = 784
hidden_shape = 128

inputs = keras.Input(shape=(input_shape,), name='digits')
x = layers.Dense(hidden_shape, activation='relu', name='dense_1')(inputs)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


batch_size = 256
train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_y))
val_dataset = val_dataset.batch(batch_size)

In [27]:
model.compile(optimizer=keras.optimizers.SGD(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()]
             )

In [28]:
model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 94s 2ms/sample - loss: 0.4642 - sparse_categorical_accuracy: 0.8660 - val_loss: 0.3108 - val_sparse_categorical_accuracy: 0.9089
Epoch 2/10
60000/60000 [==============================] - 3s 58us/sample - loss: 0.2829 - sparse_categorical_accuracy: 0.9177 - val_loss: 0.2508 - val_sparse_categorical_accuracy: 0.9271
Epoch 3/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.2351 - sparse_categorical_accuracy: 0.9333 - val_loss: 0.2103 - val_sparse_categorical_accuracy: 0.9401
Epoch 4/10
60000/60000 [==============================] - 3s 51us/sample - loss: 0.2004 - sparse_categorical_accuracy: 0.9428 - val_loss: 0.1853 - val_sparse_categorical_accuracy: 0.9460
Epoch 5/10
60000/60000 [==============================] - 3s 56us/sample - loss: 0.1761 - sparse_categorical_accuracy: 0.9499 - val_loss: 0.1669 - val_sparse_categorical_accuracy: 0.9522
Epoch 6/10
6000

## Создание новых слоев для keras при помощи tensorflow

> Наверное, одно из самых важных применений tensorflow

### Простой пример

([И ссылка на туториал](https://keras.io/layers/writing-your-own-keras-layers/))

In [29]:
class Linear(layers.Layer):

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Eager mode

> Что, если работатьс графом и сессией неудобно и не нужно? Есть *eager mode*!

[Официальный гайд](https://www.tensorflow.org/guide/eager)

In [30]:
import tensorflow as tf
tf.enable_eager_execution()

ValueError: tf.enable_eager_execution must be called at program startup.

**Проверка статуса:**

In [3]:
tf.executing_eagerly()

True

### Вычисления

> Теперь, чтобы вычислить значения, не нужно создавать сессии, нужно "всего лишь" вызвать метод `numpy()`

In [9]:
a = tf.constant(2)
b = tf.constant(2)

c = (a * b).numpy()
print(c)

4


## Вычисление градиента

> Для вычисления градиента в таком режиме используется класс `tf.GradientTape`

**Важные факты по gradient tape**

* Записывает историю вычислений во время forward pass'a и переиспользует для вычисления градиента
* Нужно создавать новый gradietn tape для каждого вычисления

In [10]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
print(grad) 

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### Оптимизация при помощи GradientTape

In [35]:
n_iter = 1000
learning_rate = 0.1

w = tf.Variable([[2.0]])
optimizer = tf.train.AdamOptimizer(learning_rate)  # [1]

for i in range(n_iter):
    with tf.GradientTape() as tape:  # [2]
        loss = 2 * w * w + w

    grad = tape.gradient(loss, w)  # [3]
    
    optimizer.apply_gradients(# zip(grad, w)  # [4]
                              [(grad, w)])    
    

In [36]:
w.numpy()

array([[-0.25]], dtype=float32)

* [1] создали оптимизатор
* [2] создали ленту
* [3] нашли градиенты (использовали ленту, на следующей итерации должна быть создана новая)
* [4] в качестве аргумента надо передавать итерируемый набор пар (градиент, тензор)